In [ ]:
import pandas as pd
import numpy as np
from causalimpact import CausalImpact
import sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_o=pd.read_csv('C:/Users/Edward/Desktop/PythonWorkspace/인허가/df_o_all.csv', encoding='cp949', index_col=0)
df_c=pd.read_csv('C:/Users/Edward/Desktop/PythonWorkspace/인허가/df_c_all.csv', encoding='cp949', index_col=0)
x = pd.read_csv('C:/Users/Edward/Desktop/PythonWorkspace/인허가/03_22_경제심리지수_금리.csv')

In [ ]:
df_c = pd.concat([df_c['폐업시기'],df_c[df_c.columns.difference(['폐업시기'])].fillna(0).astype(int)],axis=1)
df_o = pd.concat([df_o['개업시기'],df_o[df_o.columns.difference(['개업시기'])].fillna(0).astype(int)],axis=1)

In [ ]:
df_c

In [ ]:
df_o

In [ ]:
x

In [ ]:
pd.set_option('display.max_row', 1000)

In [ ]:
df_c['폐업시기'] = df_c['폐업시기'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))
df_o['개업시기'] = df_o['개업시기'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))
x['폐업시기'] = x['폐업시기'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))
x['개업시기'] = x['개업시기'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

In [ ]:
xo = x[['개업시기', '경제심리지수', '대출금리', '뉴스심리지수']]
xo

In [ ]:
xc = x[['폐업시기', '경제심리지수', '대출금리', '뉴스심리지수']]
xc

In [ ]:
df_c

In [ ]:
df_o

In [ ]:
xc

In [ ]:
xo

In [ ]:
df_c = df_c.set_index('폐업시기')
df_o = df_o.set_index('개업시기')
xc = xc.set_index('폐업시기')
xo = xo.set_index('개업시기')

In [ ]:
df_c = df_c.resample('M').agg(np.sum)
df_o = df_o.resample('M').agg(np.sum)
pre_period = ['2020-01-31', '2022-04-30']
post_period = ['2022-05-31', '2022-12-31']

In [ ]:
df_c

In [ ]:
df_o

In [ ]:
df_c = df_c.query("폐업시기 >= '2020-01-31' and 폐업시기 <='2022-12-31'")

In [ ]:
df_c

In [ ]:
df_o = df_o.query("개업시기 >= '2020-01-31' and 개업시기 <='2022-12-31'")

In [ ]:
df_o

## 분석결과

### 개업

In [ ]:
for index, row in df_o.iteritems():
    try:
        if row.sum() > 2:
            op = pd.DataFrame(row).join(xo, how='left')
        ci_o = CausalImpact(op, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        print(index)
        print(ci_o.summary())
        ci_o.plot()
        print('____________________________________________________________________________________________________')
    except ValueError as e:
        print(e)

### 폐업

In [ ]:
for index, row in df_c.iteritems():
    try:
        if row.sum() > 2:
            cl = pd.DataFrame(row).join(xc, how='left')
        ci_c = CausalImpact(cl, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        print(index)
        print(ci_c.summary())
        ci_c.plot()
        print('____________________________________________________________________________________________________')
    except ValueError as e:
        print(e)

## 결과 summary화

### 결과

In [ ]:
pre_period = ['2020-01-31', '2022-02-28']
post_period = ['2022-03-31', '2022-12-31']

In [ ]:
for index, row in df_o.iteritems():
    try:
        if row.sum() > 2:
            op = pd.DataFrame(row).join(xo, how='left')
        ci_o = CausalImpact(op, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = ci_o.summary_data.to_dict()
        print(index + "," + str(round(summary['average'].get('abs_effect'),2)) + '%,' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(ci_o.p_value,2)))
    except ValueError as e:
        print(e)

In [ ]:
for index, row in df_c.iteritems():
    try:
        if row.sum() > 2:
            cl = pd.DataFrame(row).join(xc, how='left')
        ci_c = CausalImpact(cl, pre_period, post_period, model_args={'fit_method': 'hmc'}, nseason=[{'period': 12}])
        summary = ci_c.summary_data.to_dict()
        print(index + "," + str(round(summary['average'].get('abs_effect'),2)) + '%,' + str(round((summary['average'].get('rel_effect')*100),2)) + '%,' + str(round(ci_c.p_value,2)))
    except ValueError as e:
        print(e)